In [0]:
import glob
import pandas as pd
from google.colab import files
from google.colab import drive
import numpy as np
import scipy as sc
from tqdm import tqdm_notebook as tqdm
import random
import matplotlib.pyplot as plt
import re
import random
import sys
import math
import warnings
import itertools
import collections
import seaborn as sns
warnings.filterwarnings('ignore')
from matplotlib.lines import Line2D

#Load in data

In [0]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Content Based
Sim_matrix_files = np.load("/content/drive/My Drive/Knab/Data/CleanData/CB_pearson.npz")
Sim_matrix = Sim_matrix_files['arr_0']
df_pop = pd.read_csv('/content/drive/My Drive/Knab/Data/CleanData/Popularity_score.csv')

#URLs
df_url = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/url_CB.csv")
df_url_CB = df_url.drop(['Unnamed: 0'], axis = 1)
df_url_CB = df_url_CB.rename(index= str, columns={'0':'URL_CB'})
df_url_CB['IndexCB'] = df_url_CB.index

#URL POP
df_pop_url = pd.merge(left = df_pop, right = df_url_CB, left_on = 'URL', right_on = 'URL_CB', how='right')[['IndexCB', 'COUNT']].fillna(0).sort_values(by = 'COUNT', ascending = False).reset_index(drop=True)

#Collaborative Filtering
P = np.load("/content/drive/My Drive/Knab/Data/CleanData/P.npz")['arr_0']
Q = np.load("/content/drive/My Drive/Knab/Data/CleanData/Q.npz")['arr_0']
rating_CF = np.dot(P, np.transpose(Q))

client_list_CF = pd.DataFrame(np.load("/content/drive/My Drive/Knab/Data/CleanData/client_list_CF.npz")['arr_0'])
item_list_CF = pd.DataFrame(np.load("/content/drive/My Drive/Knab/Data/CleanData/item_list_CF.npz")['arr_0'])
item_list_CF = item_list_CF.rename(index= str, columns={0:'URL_CF'})
item_list_CF['IndexCF'] = item_list_CF.index

#Click data
df_clicked = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/ALS_inputs/clicked_pairs.csv")
df_read = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/ALS_inputs/read_pairs.csv")

In [0]:
#Find combined clientlist
clicked_clients = df_clicked['clientid_hashed'].unique()
read_clients = df_read['clientid_hashed'].unique()
client_list = np.unique(np.hstack((read_clients, clicked_clients)))

In [0]:
#Popularity
pop_index = np.zeros([len(df_pop),2])
for i in range(len(df_pop)):
    pop_index[i,0] = int(df_url[df_url['0'] == df_pop.iloc[i].values[1]].index[0])
    pop_index[i,1] = df_pop.iloc[i].values[3]
pop_index = pop_index[pop_index[:,0].argsort()]
weight_list = np.transpose(np.vstack((np.arange(0,len(df_url)), np.ones([len(df_url)]))))

In [0]:
def weight_matrix(weight):
    for i in range(len(pop_index)):
        index = int(pop_index[i,0])
        weight_list[index, 1] = 1 + weight*pop_index[i,1]
#     weight_matrix = np.tile(weight_list[:,1], (len(weight_list),1))
    return weight_list[:,1]

In [0]:
def log_function(A,K,B,M,x):
    Y = A + (K-A)/((1+np.exp(-B*(x-M))))
    return Y

In [0]:
def subset_index(client):
    subset_clicked = []
    subset_read = []
    subset_clicked_CB = []
    subset_clicked_CF = []
    subset_read_CB = []
    subset_read_CF = []
    #From click-users, find the index for CB and CF
    if client in df_clicked['clientid_hashed'].unique():
        subset_clicked_URL = (df_clicked[df_clicked['clientid_hashed'] == client]['URL'].unique())
        for i in range(len(subset_clicked_URL)):
            url = subset_clicked_URL[i]
            index_CB = df_url_CB[df_url_CB[df_url_CB.columns[0]] == url].index[0]
            subset_clicked_CB.append(int(index_CB))
    
            if len(item_list_CF[item_list_CF[item_list_CF.columns[0]] == url].index) != 0:
                index_CF = item_list_CF[item_list_CF[item_list_CF.columns[0]] == url].index[0]
                subset_clicked_CF.append(int(index_CF))
                
    #From read-users, find the index for CB and CF  
    
    if client in df_read['clientid_hashed'].unique():
        subset_read_URL = (df_read[df_read['clientid_hashed'] == client]['URL'].unique())
        for i in range(len(subset_read_URL)):
            url = subset_read_URL[i]
            index_CB = df_url_CB[df_url_CB[df_url_CB.columns[0]] == url].index[0]
            subset_read_CB.append(int(index_CB))
            if len(item_list_CF[item_list_CF[item_list_CF.columns[0]] == url].index) != 0:
                index_CF = item_list_CF[item_list_CF[item_list_CF.columns[0]] == url].index[0]
                subset_read_CF.append(int(index_CF))
        
    return [[subset_read_CB, subset_read_CF], [subset_clicked_CB, subset_clicked_CF]]

In [0]:
#Find rows of sim matrix for subset
def subset_CB(df_sm, subset):
    subset_read = subset[0][0]
    subset_clicked = subset[1][0]
    subset = np.hstack((subset_read, subset_clicked))
    subset_cb = df_sm[subset.astype(int)]
    
    return subset_cb

In [0]:
#Find rows of P*Q matrix for subset
def subset_CF(rating_CF, client_list_CF, item_list_CB, item_list_CF, client, size):
    
  if client in client_list_CF[client_list_CF.columns[0]].values:
    index = client_list_CF[client_list_CF[client_list_CF.columns[0]] == client].index[0]
    subset_CF = rating_CF[index]

    merge = pd.merge(left=df_url_CB, right = item_list_CF, left_on='URL_CB', right_on='URL_CF', how='left')
    index_list = merge[['IndexCF', 'IndexCB']]

    subset_total = np.zeros(size)
    for i in range(len(index_list)):
        if str(index_list.values[i][0]) != 'nan':
            subset_total[int(index_list.values[i][1])] = subset_CF[int(index_list.values[i][0])]
        else:
            subset_total[int(index_list.values[i][1])] = 0.


    return subset_total

  else:
    return int(0)  

In [0]:
def baseline_acc(client):
  subset_indices = subset_index(client)
  
  #Only CB index
  read_set = [i[0:np.shape(subset_indices[0])[-1]-1] for i in subset_indices[0]][0]
  click_set = subset_indices[1][0]
  train_set = read_set + click_set
  test_set = subset_index(client)[0][0][-1]
  if test_set in train_set:
    train_set.remove(test_set)
  cleared = df_pop_url[~df_pop_url['IndexCB'].isin(train_set)].reset_index(drop=True)  
  k = cleared.index[cleared['IndexCB'] == str(test_set)][0] + 1
  
  return k

In [0]:
#MAIN LOOP
def total_recommendations(client, decay_penalty, clicked_penalty, weight, no_clicks, k, evaluation = False):
    #Subset indices
    subset_indices = subset_index(client)
    
    
    if evaluation == True:
      train_set = [i[0:np.shape(subset_index(client)[0])[-1]-1] for i in subset_index(client)[0]]
      subset_indices = [train_set,subset_index(client)[-1]]
      test_set = subset_index(client)[0][0][-1] #Last read of CB
      train_subset = [subset_indices[0][0], subset_indices[1][0]]
    else:
      test_set = []
      train_subset = []
    
    #Subset CB
    subset_cb = subset_CB(df_sm = Sim_matrix, subset = subset_indices)
    pop_weight = weight_matrix(weight)
    popularity = np.tile(pop_weight, (len(subset_cb) ,1))
      
    #Subset CF
    CF_row = subset_CF(rating_CF = rating_CF, client_list_CF = client_list_CF, item_list_CB = df_url_CB, \
                   item_list_CF = item_list_CF, client = client, size = len(Sim_matrix))
    
    #Dummy novelty
    D_novelty = 0.
    if type(CF_row) != int:
        D_novelty = 1
    
    #Total CB
    CB = subset_cb * pow(popularity, 1 - 2*D_novelty)
    
    #Total CF
    CF = np.tile(CF_row, (len(CB) ,1))
    
    #Confidence CF
    #Hyperparamters
    lower_bound = 1
    upper_bound = 2
    midpoint = 20
    growth_rate = 0.25
    #Calculate confidence
    CF_confidence = log_function(A = lower_bound, K = upper_bound, B = growth_rate, \
                                 M = midpoint, x = len(subset_indices[0][0]))
    
    #Total rating matrix
    rating_matrix = CB + CF*CF_confidence
    
    #Pathing weights
    weights = np.ones(len(subset_cb))
    for i in range(len(subset_indices[0][0])):
        weights[i] = 1 - i*decay_penalty
    weights[len(subset_indices[0][0]):] = clicked_penalty 
    weights[:len(subset_indices[0][0])] = weights[:len(subset_indices[0][0])][::-1] #Turn around the read pathing (last is most current)
    
    #Ratings vector
    ratings = np.average(rating_matrix, axis=0, weights = weights)
    
    #Dummy read
    if no_clicks == True:
        total_dummy = np.hstack((subset_indices[0][0], subset_indices[1][0]))
    else: 
        total_dummy = subset_indices[0][0]
    dummy_list = np.ones(len(ratings))
    dummy_list[total_dummy] = 0.
    
    #Find top K-recommendations
    rec_ratings = ratings*dummy_list #Remove read/clicked articles
    
    return [rec_ratings.argsort()[-k:][::-1], test_set, train_subset]

In [0]:
def acc_at_k(k):
  counter_prop = 0
  counter = 0
  for i in range(len(recommendations)):
    if recommendations[i][1] in recommendations[i][0][:k]:
      #1/propensity_score
      index = recommendations[i][1]
      counter_prop += 1/P_i[index,1]
      counter += 1

  accuracy = counter/len(recommendations)
  accuracy_prop = counter_prop/len(recommendations)
  return accuracy, accuracy_prop

#Evaluation

In [0]:
#Get client list +1 - clicks
df = pd.read_csv("/content/drive/My Drive/Knab/Data/CleanData/ALS_inputs/read_pairs.csv")
df = df[['URL', 'clientid_hashed']].drop_duplicates()
df_count = df['clientid_hashed'].value_counts().reset_index().rename(columns = {'index':'clientid_hashed', 'clientid_hashed':'count'})

In [0]:
client_list = df_count[(df_count['count'] < 7) & (df_count['count'] >= 2)]['clientid_hashed'] #(df_count['count'] < 7) & 

In [0]:
#Calculate propensity score
df_pop = df_pop[['URL', 'COUNT']]
df_prop = pd.merge(left=df_pop, right=df_url_CB, left_on='URL', right_on = 'URL_CB', how='right')[['IndexCB', 'COUNT']]
df_prop['IndexCB'] = df_prop['IndexCB'].astype(int)
df_prop = df_prop.sort_values(['IndexCB'])
df_prop = df_prop.fillna(0)

In [0]:
n_i = df_prop.values

In [0]:
gamma = 0.5
power = (gamma + 1)/2
P_i = n_i
P_i[:,1] = pow(P_i[:,1], power)
P_i[:,1] /=  P_i[:,1].mean()

In [0]:
random.seed(sum([ord(c) for c in "KNAB"]))
recommendations = []
decay_penalty = 0.0 #Read path
clicked_penalty = 1 #Clicked
weight = 1 #Popularity
no_clicks = False #Remove clicks from recommendations if True


#Evaluation
k = 10
eval_perc = 1

for client in tqdm(random.sample(list(client_list), int(len(client_list)*eval_perc))):
    recs = total_recommendations(client, decay_penalty, clicked_penalty, weight, no_clicks = False, k=10, evaluation = True)
    recommendations.append(recs)

In [0]:
recs = [i[0] for i in recommendations]
test = [i[1] for i in recommendations]

In [0]:
k_list = np.arange(1,11)
accuracy_list = []
accuracy_list_prop = []

In [0]:
for k in k_list:
  acc, acc_prop = acc_at_k(k)
  accuracy_list.append(acc)
  accuracy_list_prop.append(acc_prop)

In [0]:
accuracy_list

In [0]:
accuracy_list_prop

In [0]:
#Run the baseline
k_list = []
eval_perc = 0.1

for client in tqdm(random.sample(list(client_list), int(len(client_list)*eval_perc))):
  k_list.append(baseline_acc(client))

In [0]:
#HR@K
k_count = []
for k in np.arange(1,11):
  k_count.append(k_list.count(k))
  

In [0]:
baseline_accuracy = np.cumsum(k_count)/len(k_list)

In [0]:
baseline_accuracy

## Plotting

In [0]:
acc_2_NP = np.array([0.12903225806451613, 0.2144212523719165, 0.2713472485768501, 0.31688804554079697, 0.3415559772296015, 0.36432637571157495, 0.3927893738140417, 0.40512333965844405, 0.4307400379506641, 0.444022770398482])
acc_2_NP_prop = np.array([0.06146795298986862, 0.1356520646921297, 0.17158271275110235, 0.21975949792748953, 0.2369184638413932, 0.2533977686897024, 0.282101587725034, 0.29011004367540166, 0.3112706869523474, 0.3181666992127627])

In [0]:
acc_2_P = np.array([0.12144212523719165, 0.1935483870967742, 0.2381404174573055, 0.2846299810246679, 0.31688804554079697, 0.34819734345351044, 0.3709677419354839, 0.3870967741935484, 0.4146110056925996, 0.4316888045540797])
acc_2_P_prop = np.array([0.08070138706296427, 0.1187819834114933, 0.14543346004957447, 0.1794436223216601, 0.20575621524307158, 0.23322819571400746, 0.2576888051302957, 0.27258132323661655, 0.2947937103071995, 0.304761680782049])

In [0]:
acc_3_NP = np.array([0.09682947729220223, 0.15081405312767782, 0.194515852613539, 0.23564695801199656, 0.2647814910025707, 0.2982005141388175, 0.32390745501285345, 0.3427592116538132, 0.35732647814910024, 0.37703513281919454])
acc_3_NP_prop = np.array([0.04380136249303078, 0.0682324634052391, 0.10810394169683836, 0.14028947636958938, 0.16563396749124468, 0.18814817937874678, 0.20419151006540473, 0.22298181941029982, 0.2313994480951637, 0.24196768344927053])

In [0]:
acc_3_P = np.array([0.06255355612682091, 0.10968294772922023, 0.14995715509854327, 0.181662382176521, 0.207369323050557, 0.2365038560411311, 0.2647814910025707, 0.2802056555269923, 0.3067694944301628, 0.3213367609254499])
acc_3_P_prop = np.array([0.020635039486419027, 0.05161236778836449, 0.09669354116962167, 0.11804200316482354, 0.13864711171486713, 0.15515697021656008, 0.1783716952424116, 0.1919175037115841, 0.20960921941251381, 0.22110970475705058])

In [0]:
acc_4_NP = np.array([0.07939339875111508, 0.13826940231935772, 0.18287243532560213, 0.21409455842997324, 0.23639607493309545, 0.2649420160570919, 0.28099910793933985, 0.29794826048171275, 0.3247100802854594, 0.3407671721677074])
acc_4_NP_prop = np.array([0.04380136249303078, 0.0682324634052391, 0.10810394169683836, 0.14028947636958938, 0.16563396749124468, 0.18814817937874678, 0.20419151006540473, 0.22298181941029982, 0.2313994480951637, 0.24196768344927053])

In [0]:
acc_4_P = np.array([0.06155218554861731, 0.1016949152542373, 0.13648528099910795, 0.1641391614629795, 0.1873327386262266, 0.21052631578947367, 0.22747546833184656, 0.2488849241748439, 0.27029438001784123, 0.28278322925958965])
acc_4_P_prop = np.array([0.044388614747632976, 0.06581614300366838, 0.08613067549405218, 0.10553582216606675, 0.1255501024312058, 0.14134142063546642, 0.16850323461827055, 0.18283849296833948, 0.21877111099853003, 0.2350658884963826])

In [0]:
acc_5_NP = np.array([0.055891238670694864, 0.10574018126888217, 0.13897280966767372, 0.16314199395770393, 0.18882175226586104, 0.21299093655589124, 0.24018126888217523, 0.256797583081571, 0.27341389728096677, 0.29607250755287007])
acc_5_NP_prop = np.array([0.02080596409498442, 0.05590596625768417, 0.07190313143608543, 0.09108476696505297, 0.10832674051250754, 0.11747147750840783, 0.13559371619040206, 0.14591276553642515, 0.16489490574956403, 0.1833453550676272])

In [0]:
acc_5_P = np.array([0.04682779456193353, 0.08006042296072508, 0.11178247734138973, 0.13293051359516617, 0.15256797583081572, 0.1691842900302115, 0.18429003021148035, 0.21148036253776434, 0.2311178247734139, 0.24924471299093656])
acc_5_P_prop = np.array([0.01674256800743791, 0.02731583721711067, 0.0387103566131178, 0.06400907397395067, 0.07501852784426118, 0.08235606463765607, 0.09735094896624483, 0.11712536444241323, 0.12719311491312277, 0.14363388200216842])

In [0]:
acc_6_NP = np.array([0.03819444444444445, 0.0625, 0.10069444444444445, 0.1423611111111111, 0.1597222222222222, 0.1875, 0.2048611111111111, 0.2152777777777778, 0.2361111111111111, 0.2569444444444444])
acc_6_NP_prop = np.array([0.010382094887686, 0.02259084476786667, 0.04976392669430016, 0.06837234722125739, 0.08849471044605721, 0.1479666769759785, 0.15907634326757664, 0.1634018288045177, 0.1723642837067529, 0.2050835332738267])

In [0]:
acc_6_P = np.array([0.041666666666666664, 0.06597222222222222, 0.09375, 0.1076388888888889, 0.1388888888888889, 0.1527777777777778, 0.1701388888888889, 0.18055555555555555, 0.19444444444444445, 0.21180555555555555])
acc_6_P_prop = np.array([0.009972973706536395, 0.018882003689978014, 0.03164833943910815, 0.054715002682937136, 0.0761307744184258, 0.13793701931812327, 0.14519617647927477, 0.16955470184457777, 0.1758807075959985, 0.20994589540713307])

In [0]:
acc_NP = np.array([0.12373907195696032, 0.20107599193006054, 0.2589105581708137, 0.29858776059179554, 0.32817753866846, 0.3597848016139879, 0.3833221250840619, 0.39878950907868194, 0.4156018829858776, 0.433759246805649])
acc_NP_prop = np.array([0.08900470493423165, 0.13387611189788468, 0.17696034090437768, 0.20547391787017544, 0.23909813258018603, 0.27776399458615814, 0.2926081649269595, 0.3023412369131348, 0.320368159802663, 0.3443640512934281])
acc_P = np.array([0.08809683927370544, 0.14660390047074648, 0.18695359784801613, 0.23201075991930062, 0.2676529926025555, 0.2925353059852051, 0.3201075991930061, 0.3369199731002018, 0.35709482178883656, 0.3772696704774714])
acc_P_prop = np.array([0.03862095009605893, 0.06466147068681843, 0.09988120474341683, 0.1445077271710926, 0.18375628046246195, 0.2161492618802004, 0.23881021567916141, 0.25139970320147775, 0.27015084242808324, 0.2905844474068277])

In [0]:
#Only readers below 7
acc_CB = np.array([0.12373907195696032, 0.20107599193006054, 0.2589105581708137, 0.29858776059179554, 0.32817753866846, 0.3597848016139879, 0.3833221250840619, 0.39878950907868194, 0.4156018829858776, 0.433759246805649])
acc_CB_prop = np.array([0.08900470493423165, 0.13387611189788468, 0.17696034090437768, 0.20547391787017544, 0.23909813258018603, 0.27776399458615814, 0.2926081649269595, 0.3023412369131348, 0.320368159802663, 0.3443640512934281])

In [0]:
#Only 7+ readers
acc_all = np.array([0.018666666666666668, 0.048, 0.06133333333333333, 0.09333333333333334, 0.10933333333333334, 0.136, 0.16533333333333333, 0.176, 0.18933333333333333, 0.20266666666666666])
acc_no_CF = np.array([0.018666666666666668, 0.034666666666666665, 0.04533333333333334, 0.058666666666666666, 0.072, 0.096, 0.12, 0.128, 0.136, 0.14933333333333335])
acc_no_CB = np.array([0.021333333333333333, 0.029333333333333333, 0.06133333333333333, 0.08, 0.10933333333333334, 0.13333333333333333, 0.16, 0.176, 0.192, 0.216])

acc_all_prop = np.array([0.02067383040688718, 0.03350507447788771, 0.03919063806957411, 0.05170357852834995, 0.05707406911379463, 0.0792600376002176, 0.11736974543521753, 0.12955384608145104, 0.13317457086118498, 0.13904212004300942])
acc_no_CF_prop = np.array([0.024315129555105803, 0.03854491289773624, 0.04280531260308116, 0.050032640099474404, 0.07408498252545484, 0.1430233356730461, 0.17324271938104513, 0.17957226093581743, 0.19550087746137385, 0.20553203881704474])
acc_no_CB_prop = np.array([0.007903664497877702, 0.011325537286558818, 0.021380547602987177, 0.026964199865716944, 0.03436104258836833, 0.04102699622519151, 0.04902566678467556, 0.0542077240980971, 0.0648338212221426, 0.07473242483809202])

In [0]:
#Only 7+ readers
acc_highCF = np.array([0.018666666666666668, 0.048, 0.06133333333333333, 0.09333333333333334, 0.10933333333333334, 0.136, 0.16533333333333333, 0.176, 0.18933333333333333, 0.20266666666666666])
acc_lowCF = np.array([0.024, 0.034666666666666665, 0.056, 0.06933333333333333, 0.08266666666666667, 0.10666666666666667, 0.12533333333333332, 0.13866666666666666, 0.15733333333333333, 0.17066666666666666])
acc_constantCF = np.array([0.021333333333333333, 0.048, 0.06133333333333333, 0.09066666666666667, 0.10666666666666667, 0.136, 0.16533333333333333, 0.176, 0.18933333333333333, 0.20533333333333334])

acc_highCF_prop = np.array([0.02067383040688718, 0.03350507447788771, 0.03919063806957411, 0.05170357852834995, 0.05707406911379463, 0.0792600376002176, 0.11736974543521753, 0.12955384608145104, 0.13317457086118498, 0.13904212004300942])
acc_lowCF_prop = np.array([0.02513403999860091, 0.030812322829796672, 0.0405527723158271, 0.04679123729673773, 0.05409766770995826, 0.0927029112311512, 0.11578506969700007, 0.12060070326863577, 0.15654759669094326, 0.16455600569403034])
acc_constantCF_prop = np.array([0.0214917299093389, 0.033505074477887704, 0.0391906380695741, 0.05115202857224255, 0.05585358973863637, 0.07870991902537845, 0.11736974543521757, 0.12989346052333736, 0.13317457086118503, 0.14128537640974026])

In [0]:
#ALL READERS
acc_hyb = np.array([0.11380057803468208, 0.1976156069364162, 0.2518063583815029, 0.2973265895953757, 0.3211705202312139, 0.35621387283236994, 0.377528901734104, 0.39378612716763006, 0.41835260115606937, 0.43533236994219654])
acc_hyb_prop = np.array([0.0658610839064543, 0.1100323370279583, 0.1488390869463945, 0.18718376128471997, 0.20230902824759167, 0.23400280543669194, 0.26290763921841015, 0.2812790204471017, 0.3037751448029688, 0.3158105635119697])

In [0]:
baseline_acc_all = np.array([0.00304636, 0.00980132, 0.01509934, 0.04066225, 0.04529801, 0.04874172, 0.05165563, 0.05562914, 0.05986755, 0.0654304])
baseline_acc_above = np.array([0.00533333, 0.008, 0.02133333, 0.04533333, 0.05333333, 0.05333333, 0.05866667, 0.064, 0.06933333, 0.072])
baseline_acc_below = np.array([0.00282372, 0.00968132, 0.01573215, 0.04316257, 0.05123033, 0.05566761, 0.05889472, 0.06171843, 0.06575232, 0.06938282])

In [0]:
df2 = pd.DataFrame({'x': np.arange(1,11), 'N = 2, NP': acc_2_NP, 'N = 2, P': acc_2_P, 'N = 2, NP, prop': acc_2_NP_prop, 'N = 2, P, prop': acc_2_P_prop})
df3 = pd.DataFrame({'x': np.arange(1,11), 'N = 3, NP': acc_3_NP, 'N = 3, P': acc_3_P, 'N = 3, NP, prop': acc_3_NP_prop, 'N = 3, P, prop': acc_3_P_prop})
df4 = pd.DataFrame({'x': np.arange(1,11), 'N = 4, NP': acc_4_NP, 'N = 4, P': acc_4_P, 'N = 4, NP, prop': acc_4_NP_prop, 'N = 4, P, prop': acc_4_P_prop})
df5 = pd.DataFrame({'x': np.arange(1,11), 'N = 5, NP': acc_5_NP, 'N = 5, P': acc_5_P, 'N = 5, NP, prop': acc_5_NP_prop, 'N = 5, P, prop': acc_5_P_prop})
df6 = pd.DataFrame({'x': np.arange(1,11), 'N = 6, NP': acc_6_NP, 'N = 6, P': acc_6_P, 'N = 6, NP, prop': acc_6_NP_prop, 'N = 6, P, prop': acc_6_P_prop})
df = pd.DataFrame({'x': np.arange(1,11), 'NP': acc_NP, 'P': acc_P, 'NP, prop': acc_NP_prop, 'P, prop': acc_P_prop})
df_Conf = pd.DataFrame({'x': np.arange(1,11), 'High Confidence': acc_highCF, 'Low Confidence': acc_lowCF, 'Constant Confidence': acc_constantCF})
df_Conf_prop = pd.DataFrame({'x': np.arange(1,11), 'High Confidence': acc_highCF_prop, 'Low Confidence': acc_lowCF_prop, 'Constant Confidence': acc_constantCF_prop})
df_CF = pd.DataFrame({'x': np.arange(1,11), 'All': acc_all, 'No CF': acc_no_CF, 'No CB': acc_no_CB})
df_CF_prop = pd.DataFrame({'x': np.arange(1,11), 'All': acc_all_prop, 'No CF': acc_no_CF_prop, 'No CB': acc_no_CB_prop})
df_base = pd.DataFrame({'x': np.arange(1,11), 'Baseline All': baseline_acc_all, 'Baseline Above': baseline_acc_above, 'Baseline Below': baseline_acc_below})
df_all =  pd.DataFrame({'x': np.arange(1,11), 'Accuracy': acc_hyb, 'SNIPS':acc_hyb_prop})
df_CB =  pd.DataFrame({'x': np.arange(1,11), 'Accuracy': acc_CB, 'SNIPS':acc_CB_prop})

In [0]:
sns.set_style("white")

fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9


plt.plot( 'x', 'N = 2, NP, prop', data=df2, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 2, P, prop', data=df2, linestyle=':', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)


plt.plot( 'x', 'N = 3, NP, prop', data=df3, linestyle='-', marker = 'v',color='maroon', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 3, P, prop', data=df3, linestyle=':', marker = 'v',color='maroon', linewidth=2, ms=10, markevery = 1)


plt.plot( 'x', 'N = 4, NP, prop', data=df4, linestyle='-', marker = 's',color='skyblue', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 4, P, prop', data=df4, linestyle=':', marker = 's', color='skyblue', linewidth=2, ms=10, markevery = 1)


plt.plot( 'x', 'N = 5, NP, prop', data=df5, linestyle='-', marker = '>', color='olive', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 5, P, prop', data=df5, linestyle=':',  marker = '>',color='olive', linewidth=2, ms=10, markevery = 1)


plt.plot( 'x', 'N = 6, NP, prop', data=df6, linestyle='-',marker = 'p',color='darkorange', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 6, P, prop', data=df6, linestyle=':', marker = 'p',color='darkorange', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline Below', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.35)
plt.ylabel('SNIPS', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

custom_lines = [Line2D([0], [0], marker='.', color='dimgrey', lw=3, ms=15),
                Line2D([0], [0], marker='v',color='maroon', lw=3, ms=15),
                Line2D([0], [0], marker='s', color='skyblue',lw=3, ms=15),
                Line2D([0], [0], marker='>', color='olive',lw=3, ms=15),
                Line2D([0], [0], marker='p', color='darkorange',lw=3, ms=15),
                Line2D([0], [0], color='red',lw=3),
                
                Line2D([0], [0], linestyle = '-', color='black', lw=1.5),
                Line2D([0], [0], linestyle = ':', color='black', lw=1.5)
                ]



plt.legend(custom_lines, ['N = 2','N = 3','N = 4','N = 5','N = 6', 'Baseline', 'No Pathing', 'Pathing'], prop={'size': 16})

In [0]:
sns.set_style("white")

fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

plt.plot( 'x', 'N = 2, NP', data=df2, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 2, P',data=df2, linestyle=':', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)


plt.plot( 'x', 'N = 3, NP', data=df3, linestyle='-', marker = 'v', color='maroon', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 3, P',data=df3,linestyle=':', marker = 'v',color='maroon', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'N = 4, NP', data=df4, linestyle='-', marker = 's', color='skyblue', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 4, P',data=df4, linestyle=':', marker = 's',color='skyblue', linewidth=2, ms=10, markevery = 1)


plt.plot( 'x', 'N = 5, NP', data=df5, linestyle='-', marker = '>', color='olive', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 5, P',data=df5, linestyle=':',  marker = '>',color='olive', linewidth=2, ms=10, markevery = 1)


plt.plot( 'x', 'N = 6, NP', data=df6, linestyle='-', marker = 'p', color='darkorange', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'N = 6, P',data=df6, linestyle=':',marker = 'p', color='darkorange', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline Below', data=df_base, linestyle='-',color='red', linewidth=3)


plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.45)
plt.ylabel('Accuracy', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

custom_lines = [Line2D([0], [0], marker='.', color='dimgrey', lw=3, ms=15),
                Line2D([0], [0], marker='v',color='maroon', lw=3, ms=15),
                Line2D([0], [0], marker='s', color='skyblue',lw=3, ms=15),
                Line2D([0], [0], marker='>', color='olive',lw=3, ms=15),
                Line2D([0], [0], marker='p', color='darkorange',lw=3, ms=15),
                Line2D([0], [0], color='red',lw=3),
                
                Line2D([0], [0], linestyle = '-', color='black', lw=1.5),
                Line2D([0], [0], linestyle = ':', color='black', lw=1.5)
                ]


plt.legend(custom_lines, ['N = 2','N = 3','N = 4','N = 5','N = 6', 'Baseline Below', 'No Pathing', 'Pathing'], prop={'size': 16})

In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}

plt.plot( 'x', 'Accuracy', data=df_CB, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'SNIPS', data=df_CB, linestyle='-', marker = '>', color='skyblue', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline Below', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.45)
plt.ylabel('Accuracy', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)



plt.legend(prop={'size': 16})

In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.plot( 'x', 'All', data= df_CF, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CF', data=df_CF, linestyle='-.', marker = 'v', color='maroon', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CB', data=df_CF, linestyle='--', marker = '>', color='skyblue', linewidth=2, ms=10, markevery = 1)
#plt.plot( 'x', 'CC',data=df_hyb, linestyle=':', marker = 's', color='darkorange', linewidth=2, ms=10, markevery = 1)
# plt.plot( 'x', 'High CF Confidence',data=df_hyb, linestyle='-', marker = 'p', color='olive', linewidth=2, ms=10, markevery = 1)
#plt.plot( 'x', 'High CF Confidence',data=df_hyb, linestyle='--', marker = 'p', color='olive', linewidth=2, ms=10, markevery = 1)
#plt.plot( 'x', 'Baseline Above', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.24)
plt.ylabel('Accuracy', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

plt.legend(prop={'size': 16})


In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.plot( 'x', 'All', data= df_hyb_prop, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CF', data=df_hyb_prop, linestyle='-.', marker = 'v', color='maroon', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CB', data=df_hyb_prop, linestyle='--', marker = '>', color='skyblue', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'CC',data=df_hyb_prop, linestyle=':', marker = 's', color='darkorange', linewidth=2, ms=10, markevery = 1)
# plt.plot( 'x', 'High CF Confidence',data=df_hyb, linestyle='-', marker = 'p', color='olive', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'High CF Confidence',data=df_hyb_prop, linestyle='--', marker = 'p', color='olive', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'Baseline', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.15)
plt.ylabel('SNIPS', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

plt.legend(prop={'size': 16})

In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.plot( 'x', 'Accuracy', data= df_all, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'SNIPS', data=df_all, linestyle='-', marker = '>', color='skyblue', linewidth=3, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.45)
plt.ylabel('Accuracy', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

plt.legend(prop={'size': 16})

In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.plot( 'x', 'High Confidence', data= df_Conf, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'Low Confidence', data= df_Conf, linestyle='-', marker = 'p', color='skyblue', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'Constant Confidence', data= df_Conf, linestyle='-', marker = '>', color='orange', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline Above', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.24)
plt.ylabel('Accuracy', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

plt.legend(prop={'size': 16})
plt.show()

In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.plot( 'x', 'High Confidence', data= df_Conf_prop, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'Low Confidence', data= df_Conf_prop, linestyle='-', marker = 'p', color='skyblue', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'Constant Confidence', data= df_Conf_prop, linestyle='-', marker = '>', color='orange', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline Above', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.18)
plt.ylabel('SNIPS', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

plt.legend(prop={'size': 16})
plt.show()

In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.plot( 'x', 'All', data= df_CF, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CB', data= df_CF, linestyle='-', marker = 'p', color='skyblue', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CF', data= df_CF, linestyle='-', marker = '>', color='orange', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline Above', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.25)
plt.ylabel('Accuracy', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

plt.legend(prop={'size': 16})
plt.show()

In [0]:
sns.set_style("white")

# Get current size


fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

params = {'legend.fontsize': 20,
          'legend.handlelength': 2}
plt.plot( 'x', 'All', data= df_CF_prop, linestyle='-', marker = '.', color='dimgrey', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CB', data= df_CF_prop, linestyle='-', marker = 'p', color='skyblue', linewidth=2, ms=10, markevery = 1)
plt.plot( 'x', 'No CF', data= df_CF_prop, linestyle='-', marker = '>', color='orange', linewidth=2, ms=10, markevery = 1)

plt.plot( 'x', 'Baseline Above', data=df_base, linestyle='-',color='red', linewidth=3)

plt.xlabel('Number of Recommendations', fontsize = 26)
plt.ylim(0,0.23)
plt.ylabel('SNIPS', fontsize = 26)
plt.grid()


plt.tick_params(labelsize = 18)

plt.legend(prop={'size': 16})
plt.show()